In [ ]:
%pylab inline

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
import torchtext

In [ ]:
def tokenizer(text):
    return text.split()

TEXT = torchtext.legacy.data.Field(sequential=True, batch_first=True, tokenize=tokenizer, lower=True)
LABEL = torchtext.legacy.data.Field(sequential=False)

In [ ]:
train, valid, test = torchtext.legacy.datasets.SST.splits(TEXT, LABEL, root='data')

In [ ]:
# To use IMDb, which is a movie review dataset with positive/negative sentiment tags, the following code is used.
# （ポジティブ/ネガティブの感情タグ付きの映画のレビューデータセットであるIMDbを用いる場合は以下のコードを用いる．）
#
# train, test = torchtext.legacy.datasets.IMDB.splits(TEXT, LABEL, root='data')

In [ ]:
# To use your own dataset, use the following code.
# This code assumes that you have train.tsv and test.tsv files in the data/my_text_dataset folder, 
# which contain tab-delimited pairs of "text" and "label" for each line.
# （自作のデータセットを用いる場合は，以下のコードを用いる．
# このコードでは，data/my_text_dataset フォルダに，タブ区切りで「テキスト」と「ラベル」を
# 1行に1組ずつ列挙した train.tsv および test.tsv のファイルがあることを想定している．）
# train, test = torchtext.legacy.data.TabularDataset.splits(path='./data/my_text_dataset',
#                                          train='train.tsv', test='test.tsv', format='tsv',
#                                          fields=[('text', TEXT), ('label', LABEL)])

In [ ]:
TEXT.build_vocab(train, max_size=25000)
# To use a pre-trained word embedding vector, use the following code.
# （事前学習済みの単語埋め込みベクトルを用いる場合は，以下のコードを用いる．）
# TEXT.build_vocab(train, vectors="glove.6B.100d")

LABEL.build_vocab(train)

In [ ]:
print(LABEL.vocab.stoi.items())

In [ ]:
print(list(TEXT.vocab.stoi.items())[:20])
print(len(TEXT.vocab))

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
trainloader, testloader = torchtext.legacy.data.BucketIterator.splits((train, test), batch_size=4, device=device)

In [ ]:
dataiter = iter(trainloader)

In [ ]:
data = dataiter.__next__()
x, y = data.text, data.label
for x_i in x:
    print(' '.join(TEXT.vocab.itos[w] for w in x_i))
print([LABEL.vocab.itos[yi] for yi in y])

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        output, (hn, cn) = self.lstm(x)
        hn = hn.squeeze(0)
        return self.fc(hn)

In [ ]:
from itertools import islice
rnn = RNN(len(TEXT.vocab), 100, 30, 3)
# If you want to use a pre-trained word embedding vector, insert the following code.
# （事前学習済みの単語埋め込みベクトルを用いる場合は，以下のコードを挿入する．）
# rnn.embedding.weight.data.copy_(TEXT.vocab.vectors)
rnn.to(device)
optimizer = optim.SGD(rnn.parameters(), lr = 0.01)
for epoch in range(10):
    sumloss = 0.0
    # In an environment with sufficient computing resources, it is better to use all the data.
    # （計算資源が十分ある環境では，全てのデータを使う方が良い）
    #for data in trainloader:  （計算資源が十分ある環境では，全てのデータを使う方が良い）
    for data in islice(trainloader, 250): # Using only 250 batches
        x, y = data.text, data.label - 1
        optimizer.zero_grad()
        a = rnn(x)
        loss = F.cross_entropy(a, y)
        loss.backward()
        optimizer.step()
        sumloss += loss.item()
    print('epoch: {}, loss: {:.4f}'.format(epoch, sumloss))

In [ ]:
testloader.sort = False
testloader.sort_within_batch = False
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        x, y = data.text, data.label - 1
        a = rnn(x)
        pred_y = torch.argmax(a, dim=1)
        correct += (pred_y == y).sum().item()
        total += pred_y.size(0)

print(correct / total)

In [ ]:
dataiter = iter(testloader)

In [ ]:
data = dataiter.__next__()
x, y = data.text, data.label
for x_i in x:
    print(' '.join(TEXT.vocab.itos[w] for w in x_i))
a = rnn(x)
pred_y = torch.argmax(a, dim=1)
print([LABEL.vocab.itos[yi + 1] for yi in pred_y])
print([LABEL.vocab.itos[yi] for yi in y])